In [1]:
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os.path
import time, itertools, re
from collections import Counter
from sklearn.metrics import make_scorer, balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,)
from sklearn.svm import SVC

/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/


In [2]:
colsLAV=['Lav-Neg_Waardering', 'Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid']
colsSQ48=['SQ48-Vijandigheid','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie', 'SQ48-Cognitieve_Klachten',
          'SQ48-Somatische_Klachten', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme', 'SQ48-Werk_Studie']
colsMHCSF=['MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB']
cols2consider=['Main-Age','Main-ED_Codes', 'EDEQ-Score']+colsLAV+colsSQ48+colsMHCSF
cols2train=['Main-Age']+colsLAV+colsSQ48+colsMHCSF
colNames=['Main-Age', 'Main-ED_Codes', 'EDEQ-Score','Lav-Neg_Waardering','Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid', 
'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 
'MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB'] 
colRename_dict=dict(zip(cols2consider, colNames))
df_adapted_combo=pd.read_csv(fileloc_data+'maskedDAIsy_MainED_Lav_SQ48_MHCSF_Visit1.tsv', sep='\t', decimal=',')
df_adapted_combo['EDtype'] = df_adapted_combo['EDtype'].astype('category')
df_adapted_combo.astype(dict(zip(colsLAV, ['float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsSQ48,['float', 'float','float','float', 'float','float','float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsMHCSF, ['float', 'float','float']))).dtypes
df_adapted_combo.rename(columns=colRename_dict, inplace=True)
adapted_combo_cols=['Main-Age','Lav-Alg_Ontevredenheid', 'Lav-Gebrek_Vertrouwdheid','Lav-Neg_Waardering',
    'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
    'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 'MHCSF-EmotionWB', 
    'MHCSF-SocialWB', 'MHCSF-PsychWB']
#df_adapted_combo['ED_Codes'] = df_adapted_combo['EDtype'].cat.codes

In [3]:
df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train'].to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test'].to_numpy()
repeated_kfolds = RepeatedStratifiedKFold(n_splits=3, n_repeats=5)

In [6]:
iters=5
pipeRF=Pipeline(steps=[('scaler',  StandardScaler()),('RF', RandomForestClassifier(criterion="gini", 
                                                            n_estimators=300, max_features=15))])
pipeKNN=Pipeline(steps=[('scaler',  StandardScaler()),('KNN', KNeighborsClassifier(n_neighbors=7,metric='minkowski'))])
pipeLDA=Pipeline(steps=[('scaler',  StandardScaler()),('LDA', LinearDiscriminantAnalysis(solver='svd'))])
pipeQDA=Pipeline(steps=[('scaler',  StandardScaler()),('QDA', QuadraticDiscriminantAnalysis())])
pipeLSVC=Pipeline(steps=[('scaler',  StandardScaler()),('LinSVC', SVC(kernel="linear", C=1))])
pipeRSVC=Pipeline(steps=[('scaler',  StandardScaler()),('RbfSVC', SVC(kernel='rbf', C=10, gamma=0.01))])
pipeLGMLVQ=Pipeline(steps=[('scaler',  StandardScaler()),
        ('LGMLVQ', LGMLVQ(distance_type='local-adaptive-squared-euclidean', prototype_n_per_class=np.array([2, 2, 2, 1, 1]),
        solver_type='lbfgs',activation_type='swish',relevance_n_components=11,relevance_localization="class"))])
pipeGMLVQ=Pipeline(steps=[('scaler',  StandardScaler()),
            ('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', prototype_n_per_class=np.array([2, 2, 2, 1, 1]),
            activation_type='identity',relevance_n_components=7, solver_type='adam'))])
pipeGNB=Pipeline(steps=[('scaler',  StandardScaler()),('GNB', GaussianNB())])
pipeClassifiers={'RF':pipeRF, 'KNN':pipeKNN, 'LDA':pipeLDA, 'QDA': pipeQDA, 'LSVC': pipeLSVC, 
                 'RSVC': pipeRSVC, 'LGMLVQ': pipeLGMLVQ, 'GMLVQ': pipeGMLVQ, 'GNB': pipeGNB}
nclasses=len(np.unique(YTrain))
labs=np.unique(YTrain)
train_conf_all, test_conf_all, classifiers_all={},{},{}
        

In [7]:
all_relevance, all_prots=np.zeros((num_prototypes, num_features, iters)), np.zeros((num_prototypes, num_features, iters))

for key, clf in pipeClassifiers.items():
    tr_cmap=np.zeros((nclasses, nclasses, iters))
    te_cmap=np.zeros((nclasses, nclasses, iters))
    clf_all=[]
    for iter in range(iters):
        clf.fit(df_train_adapted,YTrain)
        clf_all.append(clf)
        train_pred_labs=clf.predict(df_train_adapted)
        test_pred_labs=clf.predict(df_test_adapted)
        tr_cmap[:,:,iter]=confusion_matrix(YTrain, train_pred_labs, normalize='true', labels=labs)
        te_cmap[:,:,iter]=confusion_matrix(YTest, test_pred_labs, normalize='true', labels=labs)
    classifiers_all[key]=clf_all
    train_conf_all[key], test_conf_all[key]=tr_cmap, te_cmap
    cmap_dict={'Training': tr_cmap, 'Test':te_cmap}
    for split_set in ['Training', 'Test']:
        print(key+'-'+split_set)    
        if split_set=='Training':
            printlabs=[]
            for lab in labs:
                if (lab==labs[2]) | (lab==labs[4]):
                    printlabs.append('    '+lab)
                else:
                    printlabs.append(lab)
            print(printlabs)
        mean_glob, std_glob=cmap_dict[split_set].mean(axis=2), cmap_dict[split_set].std(axis=2)
        disp_num_glob=np.reshape(np.stack((mean_glob, std_glob)).ravel('F'),(nclasses,2*nclasses))
        #disp_num_glob=[disp_num_glob]
        for idx in range(nclasses):
            cmap_cell=[]
            for jdx in range(nclasses):
                cmap_cell.append('%.3f (%.3f) '%(mean_glob[idx,jdx], std_glob[idx,jdx]))
            print(cmap_cell)    

RF-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED', '    Others']
['1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ']
RF-Test
['0.654 (0.042) ', '0.062 (0.019) ', '0.015 (0.019) ', '0.177 (0.039) ', '0.092 (0.019) ']
['0.164 (0.036) ', '0.655 (0.036) ', '0.000 (0.000) ', '0.182 (0.000) ', '0.000 (0.000) ']
['0.600 (0.045) ', '0.091 (0.000) ', '0.000 (0.000) ', '0.200 (0.036) ', '0.109 (0.068) ']
['0.188 (0.024) ', '0.235 (0.000) ', '0.012 (0.024) ', '0.329 (0.029) ', '0.235 (0.000) ']
['0.462 (0.000) ', '0.077 (0.000) ', '0.000 (0.000) ', '0.077 (0.000) ', '0.385